In [7]:
import os
import shutil
import numpy as np
import pandas as pd
import psycopg2

# Find CSV files in my current working directory
csv_files = [file for file in os.listdir(os.getcwd()) if file.endswith('.csv')]

# Create a new directory
dataset_dir = 'datasets'
try:
    os.mkdir(dataset_dir)
except FileExistsError:
    pass

# Move the CSV files to the new directory
for csv in csv_files:
    src_path = os.path.abspath(csv)
    dest_path = os.path.join(dataset_dir, csv)
    try:
        shutil.move(src_path, dest_path)
        print(f"Moved {csv} to {dataset_dir}")
    except FileNotFoundError:
        print(f"File not found: {csv}")
    except shutil.Error as e:
        print(f"Error moving file {csv}: {e}")

# Read CSV files into Pandas DataFrames
df = {}
for file in csv_files:
    file_path = os.path.join(dataset_dir, file)
    try:
        df[file] = pd.read_csv(file_path, encoding="ISO-8859-1")
        print(f"Using ISO-8859-1 encoding for {file}")
    except UnicodeDecodeError:
        print(f"Error decoding file {file}")

# Clean table names and column names
for file, dataframe in df.items():
    clean_table_name = file.lower().replace(" ", "_").replace("?", "") \
        .replace("-", "_").replace("/", "_").replace("\\", "_").replace("%", "") \
        .replace(")", "").replace("(", "").replace("$", "")

# Remove .csv extension from clean_table_name
    table_name = clean_table_name.split('.')[0]

    dataframe.columns = [col.lower().replace(" ", "_").replace("?", "").replace("¢", "") \
                         .replace("-", "_").replace("/", "_").replace("\\", "_").replace("%", "") \
                         .replace(")", "").replace("(", "").replace("$", "").replace(".", "")
                         for col in dataframe.columns]

   # Replacement dictionary that maps pandas datatypes to PostgreSQL datatypes
    replacements = {
    'object': 'varchar',
    'float64': 'double precision',
    'float32': 'real',
    'int64': 'bigint',
    'int32': 'integer',
    'int16': 'smallint',
    'timedelta64[ns]': 'varchar',
    'timedelta64': 'interval',
    'datetime64[ns]': 'timestamp',
    'bool': 'boolean',
    'datetime64': 'timestamp'}
    
    # Table Schema
    col_str = ", ".join("{} {}".format(n, replacements.get(str(d), 'varchar')) for (n, d) in
                        zip(dataframe.columns, dataframe.dtypes))

    # Open a database connection
    hostname = 'localhost'
    database = 'globalsuperstore'
    username = 'postgres'
    password = 'password'
    port_id = 5432
    
    conn = psycopg2.connect(
        host=hostname,
        dbname=database,
        user=username,
        password=password,
        port=port_id
    )
    cursor = conn.cursor()
    print(f'Opened the database successfully for {table_name}')

    # Drop table with same name
    cursor.execute(f"DROP TABLE IF EXISTS {table_name};")
    
    # Create SQL table
    cursor.execute(f"CREATE TABLE {table_name} ({col_str});")
    print(f'{table_name} was created successfully')

    # Save dataframe to csv file
    dataframe.to_csv(file, header=dataframe.columns, index=False, encoding='utf-8')

    # Open csv file,save it as an object
    with open(file) as my_file:
        print(f'File {file} opened in memory')
        
    # Upload to database    
        cursor.copy_expert(sql=f"COPY {table_name} FROM STDIN WITH CSV HEADER DELIMITER AS ',';", file=my_file)
        print(f'File {file} copied to database')

    # Open permission for any user that access the database
    cursor.execute(f"GRANT SELECT ON TABLE {table_name} TO public;")
    conn.commit()
    cursor.close()
    conn.close()

# For loop end message
print('All tables have been successfully imported into the database.')


Moved Customers.csv to datasets
Moved Orders.csv to datasets
Moved Products.csv to datasets
Moved Returns.csv to datasets
Using ISO-8859-1 encoding for Customers.csv
Using ISO-8859-1 encoding for Orders.csv
Using ISO-8859-1 encoding for Products.csv
Using ISO-8859-1 encoding for Returns.csv
Opened the database successfully for customers
customers was created successfully
File Customers.csv opened in memory
File Customers.csv copied to database
Opened the database successfully for orders
orders was created successfully
File Orders.csv opened in memory
File Orders.csv copied to database
Opened the database successfully for products
products was created successfully
File Products.csv opened in memory
File Products.csv copied to database
Opened the database successfully for returns
returns was created successfully
File Returns.csv opened in memory
File Returns.csv copied to database
All tables have been successfully imported into the database.
